## Benchmark Question-Answer Generation

This notebook demonstrates how we can generate a set of Questions and Answers based on chunks from a database. Documents chunked before insertion to a database and saved to `.csv` for MSFT transcripts sample. It is beneficial to generate questions and answers at this stage, as doing a search from a database has an added cost. Randomly picking chunks for specified filter parameters ensures diversity of the questions and answers. 

### SET UP AND CONFIGURATION

Load Environment File

In [1]:
from dotenv import dotenv_values

# specify the name of the .env file name 
env_name = "../../.env" # change to your own .env file name
config = dotenv_values(env_name)

Read Chunks from csv (see step2 notebook from preprocessing subdirectory)

In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv('AnalyzedPDF/ChunksEmbedding.csv')

In [3]:
df[df['Quarter']==2]

,Id,Ticker,Year,Quarter,Chunk,PageNumber,LineNumber,Embedding
113,114,MSFT,23,2,Microsoft FY23 Second Quarter Earnings Confere...,1,1,"[-0.022043932, -0.023832329, -0.015447599, -0...."
114,115,MSFT,23,2,"On the Microsoft Investor Relations website, y...",1,9,"[-0.023697682, -0.005627374, -0.0051322975, -0..."
115,116,MSFT,23,2,GAAP. They are included as additional clarifyi...,1,17,"[-0.012550131, -0.0020706053, 0.007283737, -0...."
116,117,MSFT,23,2,"same in constant currency, we will refer to th...",1,25,"[-0.01768585, -0.02943631, -0.00054391, -0.015..."
117,118,MSFT,23,2,"predictions, projections, or other statements ...",2,6,"[-0.009156934, -0.019673413, -0.0082705645, -0..."
...,...,...,...,...,...,...,...,...
216,217,MSFT,23,2,"BRETT IVERSEN: Thanks, Brad. Joe, we have time...",31,13,"[0.0011864604, -0.04014092, 0.009777045, -0.01..."
217,218,MSFT,23,2,the coming quarters? Thank you. SATYA NADELLA:...,31,21,"[0.009347212, -0.01008223, 0.015255094, -0.009..."
218,219,MSFT,23,2,"going to be an Al app. That's, I think, the be...",32,1,"[0.005495851, -0.003575635, 0.013053961, 0.000..."
219,220,MSFT,23,2,"Sometimes, you will have ISVs who are differen...",32,9,"[-0.004339527, -0.028531296, 0.017532898, -0.0..."


In [4]:
df['Chunk'].iloc[11]

"Cosmos DB now supports PostgreSQL, making Azure the first cloud provider to offer a database service that supports both relational and NoSQL workloads. And, in Al, we are turning the world's most advanced models into platforms for customers. Earlier this month, we brought the power of Dall-E to Azure OpenAI service, helping customers like Mattel apply the breakthrough image generation model to commercial use cases for the first time. And Azure Machine Learning provides industry leading MLOps, helping organizations like 3M deploy, manage, and govern models. "

#### Prompt Template
##### Write a Prompt Tempalte. The prompt template should include all filter keys, so they can be referenced and input.

In [75]:
template = """
        You are given two chunks of text, a ticker e.g. MSFT, Quarter, Year, as input. You will generate 10 relevant questions and answers pairs based on the input.
        The question should be formed based on information in both the chunks of text.
        The answers should be available in the two chunks of text. Do not generate answers on your own.  If answer is not available in the text, just write N/A.
               
        An example output for this example is: 

        Question: For {ticker} FY{year} Q{quarter}, what is the <question goes here>?
        Answer: example answer paraphrased from the relevant information in the given text goes here 

        Based on ticker, quarter, year, the question can be phrased in different ways e.g. MSFT FY23 Q1, MSFT FY2023 1st quarter, e.t.c.
        In case the text question is not relevant, please skip the question and answer pair.
        input_text1: 
        {chunk_text1}
        input_text2:
        {chunk_text2}
        ticker: {ticker}
        quarter: {quarter}
        year: {year}
        """

#### Randomly pick filter parameters:

Add Filter Parameters to randomly pick them for extracting context (chunks). This will help diversify generating questions and answers. 
In the MSFT Financial Transcripts use-case, the Ticker name is MSFT, but you can easily add other ticker labels for a larger Financial dataset. `Year`,`Quarter`, and `Id` are the key parameters used in this use-case. In this notebook, we are using two random chunks (chunk ids) from specified filter keys (year, quarter).

###### TODO: Add Tools (an updated version of function calls) once it is available for working with newer models. Function Calls are currently deprecated for gpt models with versions beyond 07-01-2023.

In [55]:
Ticker = np.random.choice(df['Ticker'].unique())
Year = np.random.choice(df[df['Ticker']==Ticker]['Year'].unique())
Quarter = np.random.choice(df[(df['Ticker']==Ticker) & (df['Year']==Year)]['Quarter'].unique())
# Id = np.random.choice(df[(df['Quarter']==Quarter) & (df['Year']==Year)]['Id'].unique())
# Id2 = np.random.choice(df[(df['Quarter']==Quarter) & (df['Year']==Year) & (df['Id']!=Id)]['Id'].unique())

In [53]:
pagenum1 = np.random.choice(df[(df['Ticker']==Ticker) & (df['Year']==Year) & (df['Quarter']==Quarter)]['PageNumber'].unique())
pagenum2 = np.random.choice(df[(df['Ticker']==Ticker) & (df['Year']==Year) & (df['Quarter']==Quarter)]['PageNumber'].unique())

In [56]:
chunk1 = df[(df['Ticker']==Ticker) & (df['Year']==Year) & (df['Quarter']==Quarter) & (df['PageNumber']==pagenum1)]['Chunk'].str.cat(sep=' ')
chunk2 = df[(df['Ticker']==Ticker) & (df['Year']==Year) & (df['Quarter']==Quarter) & (df['PageNumber']==pagenum2)]['Chunk'].str.cat(sep=' ')

In [60]:
len(chunk1)

1214

In [61]:
chunk1

"And Azure AI is ushering in new, born-in-the cloud AI-first workloads, with the best selection of frontier and open models, including Meta's recent announcement supporting Llama on Azure and Windows, as well as OpenAI. We have great momentum across Azure OpenAI Service. More than 11,000 organizations across industries, including Ikea, Volvo Group, Zurich Insurance, as well as digital natives like Flipkart, Humane, Kahoot, Miro, Typeface, use the service. That's nearly 100 new customers added every day this quarter. Mercedes-Benz, for example, is bringing ChatGPT via Azure OpenAI to  That's nearly 100 new customers added every day this quarter. Mercedes-Benz, for example, is bringing ChatGPT via Azure OpenAI to more than 900,000 vehicles in the United States, making its in-car voice assistant more intuitive. And Moody's built its own internal copilot to improve productivity of its 14,000 employees. We are also partnering broadly to scale this next generation of AI to more customers. Sn

In [58]:
print(Ticker, Year, Quarter, Id, Id2)

MSFT 23 4 357 418


#### Generate Questions (using Azure OpenAI only)

In [76]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
  api_key = config["OPENAI_API_KEY"],  
  api_version = config["OPENAI_API_VERSION"],
  azure_endpoint = config["OPENAI_API_BASE"]
)

response = client.chat.completions.create(
    model="gpt-35-turbo", # model = "deployment_name".
    messages=[
        {"role": "system", "content":"You are a generator of questions and answers for the given text." },
        {"role": "user", "content": template.format(chunk_text1=chunk1, chunk_text2=chunk2, 
                                                    ticker=Ticker, year=str(Year), quarter=str(Quarter))}
    ]
)

#print(response)
# print(response.model_dump_json(indent=2))
print(response.choices[0].message.content)

#TODO: Add cells showing adding tools to the chat completion. It is an update to functionc calling feature. Function calling feature is not available in the current version of the API.OpenAI version > 1.0.0.

Question 1: What AI models are supported by Azure AI for MSFT FY23 Q4?
Answer 1: Azure AI supports the best selection of frontier and open models, including Meta's recent announcement supporting Llama on Azure and Windows, as well as OpenAI.

Question 2: How many new customers were added to Azure OpenAI service during MSFT FY23 Q4?
Answer 2: Nearly 100 new customers were added every day during MSFT FY23 Q4, including organizations across industries such as Ikea, Volvo Group, Zurich Insurance, Flipkart, Humane, Kahoot, Miro, and Typeface.

Question 3: Which organization is bringing ChatGPT via Azure OpenAI to over 900,000 vehicles in the United States for MSFT FY23 Q4?
Answer 3: Mercedes-Benz is bringing ChatGPT via Azure OpenAI to more than 900,000 vehicles in the United States during MSFT FY23 Q4.

Question 4: Who built their own internal copilot to improve productivity of its 14,000 employees in MSFT FY23 Q4?
Answer 4: Moody's built its own internal copilot to improve productivity of

In [82]:
print(response.choices[0].message.content)


Question 1: What AI models are supported by Azure AI for MSFT FY23 Q4?
Answer 1: Azure AI supports the best selection of frontier and open models, including Meta's recent announcement supporting Llama on Azure and Windows, as well as OpenAI.

Question 2: How many new customers were added to Azure OpenAI service during MSFT FY23 Q4?
Answer 2: Nearly 100 new customers were added every day during MSFT FY23 Q4, including organizations across industries such as Ikea, Volvo Group, Zurich Insurance, Flipkart, Humane, Kahoot, Miro, and Typeface.

Question 3: Which organization is bringing ChatGPT via Azure OpenAI to over 900,000 vehicles in the United States for MSFT FY23 Q4?
Answer 3: Mercedes-Benz is bringing ChatGPT via Azure OpenAI to more than 900,000 vehicles in the United States during MSFT FY23 Q4.

Question 4: Who built their own internal copilot to improve productivity of its 14,000 employees in MSFT FY23 Q4?
Answer 4: Moody's built its own internal copilot to improve productivity of

In [77]:
df[(df['Quarter']==Quarter) & (df['Year']==Year) & (df['Id']==Id)].Chunk.values[0]

"Hyperverge. Now, on to search, advertising, and news. While it's early in our journey, we are reshaping daily search and web habits with our Copilot for the web. This quarter, we introduced new AI-powered features, including multimodal capabilities with visual search in Bing Chat. We're expanding to businesses, with Bing Chat Enterprise, which offers commercial data protection, providing an easy on-ramp for any organization looking to get the benefit of this next generation of AI today. Bing is also the default search experience for OpenAl's ChatGPT, bringing "

In [78]:
import re
_pattern = re.compile(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s')
retrieved_sentences=_pattern.split(chunk1)
retrieved_sentences

["And Azure AI is ushering in new, born-in-the cloud AI-first workloads, with the best selection of frontier and open models, including Meta's recent announcement supporting Llama on Azure and Windows, as well as OpenAI.",
 'We have great momentum across Azure OpenAI Service.',
 'More than 11,000 organizations across industries, including Ikea, Volvo Group, Zurich Insurance, as well as digital natives like Flipkart, Humane, Kahoot, Miro, Typeface, use the service.',
 "That's nearly 100 new customers added every day this quarter.",
 "Mercedes-Benz, for example, is bringing ChatGPT via Azure OpenAI to  That's nearly 100 new customers added every day this quarter.",
 'Mercedes-Benz, for example, is bringing ChatGPT via Azure OpenAI to more than 900,000 vehicles in the United States, making its in-car voice assistant more intuitive.',
 "And Moody's built its own internal copilot to improve productivity of its 14,000 employees.",
 'We are also partnering broadly to scale this next generatio

In [79]:
retrieved_sentences2=_pattern.split(chunk2)
retrieved_sentences2

['course, be able to sell it and then recognize revenue.',
 'And that is why I continue to say that I am just as excited as everyone else about this, and it should be more H2 weighted.',
 "And we've, I think, given you some sizing opportunities and I think I would use all that.",
 'But I do think this is really about pacing.',
 "And, of course, we've still got to get our Security Copilot and some of the Dynamics workloads priced and released, and we'll continue to work toward that.",
 "And, of course, I think one of the things that people often, I think, overlook is, and Satya mentioned it briefly, when you go back to the pull along Azure, I think in many ways, lots of these AI products pull along Azure  is, and Satya mentioned it briefly, when you go back to the pull along Azure, I think in many ways, lots of these AI products pull along Azure because it's not just the Al solution services that you need to build an app.",
 "And so, it's less about Microsoft 365 pulling it along or any

In [83]:
response.choices[0].message.content

"Question 1: What AI models are supported by Azure AI for MSFT FY23 Q4?\nAnswer 1: Azure AI supports the best selection of frontier and open models, including Meta's recent announcement supporting Llama on Azure and Windows, as well as OpenAI.\n\nQuestion 2: How many new customers were added to Azure OpenAI service during MSFT FY23 Q4?\nAnswer 2: Nearly 100 new customers were added every day during MSFT FY23 Q4, including organizations across industries such as Ikea, Volvo Group, Zurich Insurance, Flipkart, Humane, Kahoot, Miro, and Typeface.\n\nQuestion 3: Which organization is bringing ChatGPT via Azure OpenAI to over 900,000 vehicles in the United States for MSFT FY23 Q4?\nAnswer 3: Mercedes-Benz is bringing ChatGPT via Azure OpenAI to more than 900,000 vehicles in the United States during MSFT FY23 Q4.\n\nQuestion 4: Who built their own internal copilot to improve productivity of its 14,000 employees in MSFT FY23 Q4?\nAnswer 4: Moody's built its own internal copilot to improve prod

### Save Eval Set to a csv


**Note on text structuring and format:** We could chain another llm call to convert to a format suitable to be saved to csv. The prompt can be modified to provide answer in this form directly. But, it is left as an exercise for the user to update prompt to work with whatever format they want to use. Here we split and reorganize the format output by the model using python, for the promptflow sample.

**Note on csv dataset format choice:** csv is a customer requirement, and a proper way can be to log them to a database and populate them. 

In [86]:
qa_string = response.choices[0].message.content
# Parse the string into rows
rows = [row.strip() for row in qa_string.split('\n') if row.strip()]

# Separate questions and answers
questions = [row.split(": ", 1)[1] for i, row in enumerate(rows) if i % 2 == 0]
answers = [row.split(": ", 1)[1] for i, row in enumerate(rows) if (i - 1) % 2 == 0]

# Combine into a list of dictionaries
qa_data = [{"chat_history": "[]", "question": q, "answer": a} for q, a in zip(questions, answers)]

In [87]:
qa_data

[{'chat_history': '[]',
  'question': 'What AI models are supported by Azure AI for MSFT FY23 Q4?',
  'answer': "Azure AI supports the best selection of frontier and open models, including Meta's recent announcement supporting Llama on Azure and Windows, as well as OpenAI."},
 {'chat_history': '[]',
  'question': 'How many new customers were added to Azure OpenAI service during MSFT FY23 Q4?',
  'answer': 'Nearly 100 new customers were added every day during MSFT FY23 Q4, including organizations across industries such as Ikea, Volvo Group, Zurich Insurance, Flipkart, Humane, Kahoot, Miro, and Typeface.'},
 {'chat_history': '[]',
  'question': 'Which organization is bringing ChatGPT via Azure OpenAI to over 900,000 vehicles in the United States for MSFT FY23 Q4?',
  'answer': 'Mercedes-Benz is bringing ChatGPT via Azure OpenAI to more than 900,000 vehicles in the United States during MSFT FY23 Q4.'},
 {'chat_history': '[]',
  'question': 'Who built their own internal copilot to improve 

In [88]:
csv_file_path = "../datasets/generated_eval_set.csv"
import csv
# Writing to CSV file
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=["chat_history", "question", "answer"])
    
    # Write headers
    writer.writeheader()
    
    # Write data
    writer.writerows(qa_data)

print(f"Data has been saved to {csv_file_path}.")

Data has been saved to ../datasets/generated_eval_set.csv.
